<a href="https://colab.research.google.com/github/congnhatdt03-hue/Nhan_dien_bien_so_xe/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_web_video_ngrok__%C4%90%C3%A3_ho%C3%A0n_thi%E1%BB%87n.%20c%E1%BA%ADp%20nh%E1%BA%ADt%20m%E1%BB%9Bi%20nh%E1%BA%A5t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q streamlit opencv-python pillow ultralytics easyocr google-generativeai cloudflared



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.3 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!mkdir -p /content/project/models
!cp "/content/drive/MyDrive/đồ án 2/best.pt" /content/project/models/
!cp "/content/drive/MyDrive/đồ án 2/sort.py" /content/project

In [6]:
!pip install filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=1b44ed2a19319039df73cf1b6642eed549b9150bd4058d2f3a95635bb5011a10
  Stored in directory: /root/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy


In [27]:
%%writefile /content/project/app.py
import streamlit as st
import cv2
import numpy as np
import re
import os
import tempfile
from PIL import Image
from ultralytics import YOLO
import easyocr
from collections import Counter
import sys
sys.path.append('/content/sort')  # Thêm nếu chưa có
from sort import Sort
import subprocess

st.set_page_config(page_title="Nhận diện biển số xe", layout="wide")
# =================== HÀM HỖ TRỢ ===================
def iou(b1, b2):
    x1, y1, x2, y2 = b1
    X1, Y1, X2, Y2 = b2
    inter_x1, inter_y1 = max(x1, X1), max(y1, Y1)
    inter_x2, inter_y2 = min(x2, X2), min(y2, Y2)
    iw, ih = max(0, inter_x2 - inter_x1), max(0, inter_y2 - inter_y1)
    inter = iw * ih
    a1 = (x2 - x1) * (y2 - y1)
    a2 = (X2 - X1) * (Y2 - Y1)
    return inter / (a1 + a2 - inter + 1e-6)

def sharpness_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# ====== CONFIG ======
YOLO_PATH = "/content/drive/MyDrive/đồ án 2/best.pt"
FRAME_STEP = 200  # ms, giảm OCR để mượt hơn
CONFIDENCE_THRESHOLD = 0.25
USE_EASYOCR_FALLBACK = True
IOU_TH = 0.6      # Match chắc hơn
MAX_AGE = 50      # Track giữ lâu hơn

# ====== GEMINI API KEY ======
import google.generativeai as genai
GEMINI_API_KEY = st.sidebar.text_input("🔑 Nhập Gemini API Key:", type="password")
if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    gemini_model = genai.GenerativeModel("gemini-1.5-flash")
else:
    gemini_model = None

# ====== LOAD YOLO MODEL ======
@st.cache_resource
def load_yolo(path):
    model = YOLO(path)
    return model
model = load_yolo(YOLO_PATH)

# ====== REGEX CHUẨN HÓA ======
FALLBACK_REGEX = re.compile(r"(\d{2})\s*[- ]?\s*([A-Z]{1,2})\s*[- ]?\s*([0-9]{2,6})")
def normalize_plate(text: str) -> str:
    text = text.upper()
    text = re.sub(r"[^A-Z0-9]", "", text)
    match = re.match(r"^(\d{2})([A-Z]{1,2})(\d{2,6})$", text)
    if match:
        p1, p2, p3 = match.groups()
        return f"{p1}-{p2} {p3}"
    match2 = FALLBACK_REGEX.search(text)
    if match2:
        p1, p2, p3 = match2.groups()
        return f"{p1}-{p2} {p3}"
    return text

# ====== OCR ======
def ocr_with_gemini(image: Image.Image) -> str:
    try:
        prompt = "Đọc biển số xe trong ảnh (chỉ trả lời biển số, không thêm ký tự nào khác):"
        res = gemini_model.generate_content([prompt, image])
        txt = res.text.strip()
        return normalize_plate(txt)
    except Exception as e:
        st.warning(f"Lỗi Gemini: {e}")
        return ""

@st.cache_resource
def load_easyocr():
    return easyocr.Reader(['en'])

def ocr_with_easyocr(image: np.ndarray) -> str:
    reader = load_easyocr()
    results = reader.readtext(image)
    txt = " ".join([r[1] for r in results])
    return normalize_plate(txt)

# ====== TIỀN XỬ LÝ ROI ======
def enhance_roi(roi: np.ndarray) -> np.ndarray:
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(gray)
    sharp = cv2.addWeighted(clahe, 1.5, cv2.GaussianBlur(clahe, (0,0), 1), -0.5, 0)
    return cv2.cvtColor(sharp, cv2.COLOR_GRAY2BGR)

# ====== HIỂN THỊ GIAO DIỆN ======
st.title("Nhận diện biển số xe từ ảnh hoặc video")
st.markdown("<div style='text-align:center; font-size:18px; color:gray;'>Đồ án II – GVHD: ThS. Nguyễn Thị Huế</div>", unsafe_allow_html=True)

input_type = st.radio("Chọn loại dữ liệu:", ["Ảnh", "Video"])

if input_type == "Ảnh":
    uploaded_file = st.file_uploader("📤 Tải lên ảnh", type=["jpg", "jpeg", "png"])
else:
    uploaded_file = st.file_uploader("📤 Tải lên video", type=["mp4", "mov", "avi"])

if uploaded_file and st.button("🚀 Bắt đầu nhận diện"):
    plate_counter = Counter()

    # ====== XỬ LÝ ẢNH ======
    if input_type == "Ảnh":
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        result = model.predict(image, conf=CONFIDENCE_THRESHOLD)[0]
        annotated = image.copy()

        if len(result.boxes) == 0:
            st.warning("❌ Không có biển số được phát hiện.")
        else:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                roi = annotated[y1:y2, x1:x2]
                roi = enhance_roi(roi)

                if gemini_model:
                    pil_roi = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
                    text = ocr_with_gemini(pil_roi)
                    if not text.strip():
                        st.info("⚠️ Gemini vượt quota → dùng EasyOCR.")
                        text = ocr_with_easyocr(roi)
                else:
                    text = ocr_with_easyocr(roi) if USE_EASYOCR_FALLBACK else ""

                plate_counter[text] += 1
                cv2.rectangle(annotated, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(annotated, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

            st.image(annotated, caption="Ảnh đã nhận diện", channels="BGR", use_column_width=True)

    # ====== XỬ LÝ VIDEO ======
    else:
        # Lưu file tạm
        temp_video = tempfile.NamedTemporaryFile(delete=False)
        temp_video.write(uploaded_file.read())
        temp_video_path = temp_video.name

        cap = cv2.VideoCapture(temp_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps * FRAME_STEP / 1000) if FRAME_STEP > 0 else 1
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out_path = "output_annotated.mp4"
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        pbar = st.progress(0.0)
        ocr_status = st.empty()

        # ================= PASS 1: Quét video, tạo track với SORT, chọn ROI sắc nét nhất =================
        sort_tracker = Sort(max_age=MAX_AGE, min_hits=1, iou_threshold=IOU_TH)
        tracks = {}  # tid -> {"best_roi": np.array, "best_score": float, "text": str, "boxes_history": list}
        box_found = False
        idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if idx % frame_interval == 0:
                result = model.predict(frame, conf=CONFIDENCE_THRESHOLD)[0]
                dets = np.array([[*box.xyxy[0].cpu().numpy(), box.conf[0].cpu().numpy()] for box in result.boxes]) if len(result.boxes) > 0 else np.empty((0, 5))
                if len(dets) > 0:
                    box_found = True
            else:
                dets = np.empty((0, 5))

            trackers = sort_tracker.update(dets)

            for track in trackers:
                x1, y1, x2, y2, tid = map(int, track)
                if tid not in tracks:
                    tracks[tid] = {
                        "best_roi": None,
                        "best_score": -1.0,
                        "text": "",
                        "boxes_history": []
                    }

                # Lưu lịch sử box để làm mượt
                tracks[tid]["boxes_history"].append([x1, y1, x2, y2])
                if len(tracks[tid]["boxes_history"]) > 7:  # Giữ 7 frame
                    tracks[tid]["boxes_history"].pop(0)

                # Cập nhật ROI sắc nét nhất
                matched = any(iou([x1, y1, x2, y2], det[:4]) > IOU_TH for det in dets)
                if matched:
                    roi_x1, roi_y1, roi_x2, roi_y2 = map(int, [x1, y1, x2, y2])
                    roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
                    if roi.size == 0:
                        continue
                    roi = enhance_roi(roi)
                    sc = sharpness_score(roi)
                    if sc > tracks[tid]["best_score"]:
                        tracks[tid]["best_score"] = sc
                        tracks[tid]["best_roi"] = roi

            idx += 1
            pbar.progress(min(idx / max(frames, 1), 1.0))

        cap.release()

        # ================= OCR một lần duy nhất cho mỗi track =================
        unique_texts = []
        for tid, t in tracks.items():
            if t["best_roi"] is not None:
                try:
                    if gemini_model:
                        ocr_status.markdown("✅ **Đang dùng Gemini OCR**", unsafe_allow_html=True)
                        pil_roi = Image.fromarray(cv2.cvtColor(t["best_roi"], cv2.COLOR_BGR2RGB))
                        txt = ocr_with_gemini(pil_roi)
                        if not txt.strip():
                            ocr_status.markdown("⚠️ **Gemini vượt quota, chuyển EasyOCR**", unsafe_allow_html=True)
                            txt = ocr_with_easyocr(t["best_roi"])
                    else:
                        ocr_status.markdown("⚠️ **Chưa có API key, dùng EasyOCR**", unsafe_allow_html=True)
                        txt = ocr_with_easyocr(t["best_roi"])
                    t["text"] = txt.strip()
                except Exception as e:
                    ocr_status.markdown(f"⚠️ **Gemini lỗi ({e}), dùng EasyOCR**", unsafe_allow_html=True)
                    t["text"] = ocr_with_easyocr(t["best_roi"])
                if t["text"]:
                    unique_texts.append(t["text"])

        # ================= PASS 2: Ghi video output với nhãn cố định, smooth box =================
        cap = cv2.VideoCapture(temp_video_path)
        sort_tracker = Sort(max_age=MAX_AGE, min_hits=1, iou_threshold=IOU_TH)
        idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if idx % frame_interval == 0:
                result = model.predict(frame, conf=CONFIDENCE_THRESHOLD)[0]
                dets = np.array([[*box.xyxy[0].cpu().numpy(), box.conf[0].cpu().numpy()] for box in result.boxes]) if len(result.boxes) > 0 else np.empty((0, 5))
            else:
                dets = np.empty((0, 5))

            trackers = sort_tracker.update(dets)

            for track in trackers:
                x1, y1, x2, y2, tid = map(int, track)
                # Match tid với tracks từ pass 1 bằng IoU
                best_tid, best_iou = None, 0.0
                for saved_tid, t in tracks.items():
                    if t["boxes_history"]:
                        boxes_np = np.array(t["boxes_history"])
                        last_box = np.mean(boxes_np, axis=0).astype(int)  # Moving average
                        ov = iou([x1, y1, x2, y2], last_box)
                        if ov > IOU_TH and ov > best_iou:
                            best_tid, best_iou = saved_tid, ov

                if best_tid is not None:
                    label = tracks[best_tid]["text"]
                    # Làm mượt box bằng moving average
                    if tracks[best_tid]["boxes_history"]:
                        boxes_np = np.array(tracks[best_tid]["boxes_history"])
                        x1, y1, x2, y2 = np.mean(boxes_np, axis=0).astype(int)
                else:
                    label = ""

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                if label:
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            out.write(frame)
            idx += 1

        cap.release()
        out.release()

        # ================= Hiển thị kết quả =================
        if not box_found:
            st.warning("❌ Không có biển số được phát hiện trong video.")
        else:
            # 1) Hiển thị video gốc (Căn giữa & thu nhỏ)
            st.subheader(" Video tải lên")
            col1, col2, col3 = st.columns([1, 2, 1])  # 2 cột trống để căn giữa video
            with col2:
                st.video(temp_video_path)
            # 2) Hiển thị ảnh ROI tốt nhất của từng biển số (nếu có)
            for tid, t in tracks.items():
                if t["best_roi"] is not None and t["text"]:
                    st.image(
                        t["best_roi"],
                        caption=f"🖼️ Track #{tid} (Biển số: {t['text']})",
                        use_column_width=True
                    )

            # 3) Nút tải video kết quả có bbox & biển số
            st.subheader("📥 Video đã nhận diện")
            with open(out_path, "rb") as f:
                st.download_button(
                    "📥 Tải video đã nhận diện",
                    f,
                    file_name="output_annotated.mp4"
                )

        # Cập nhật danh sách biển số đã nhận diện
        for txt in unique_texts:
            plate_counter[txt] += 1

    # ===== KẾT QUẢ CUỐI =====
    st.subheader("📋 Biển số phát hiện được:")
    found_plate = False
    for plate, count in plate_counter.items():
        if plate.strip():
            st.write(f"- **{plate}** ({count} lần)")
            found_plate = True
    if not found_plate:
        st.warning("❌ Không có biển số được phát hiện.")

# ===== GHI CHÚ =====
    st.markdown(
        """
        <div style='position: fixed; bottom: 10px; right: 10px; font-size: 14px; color: gray;'>
           By Võ Công Nhật-20222627.
        </div>
        """,
        unsafe_allow_html=True
    )

# Dọn file tạm
if 'temp_video' in locals() and os.path.exists(temp_video.name):
    os.unlink(temp_video.name)
if 'out_path' in locals() and os.path.exists(out_path):
    os.unlink(out_path)

Overwriting /content/project/app.py


In [28]:
!streamlit run /content/project/app.py --server.port 8501 &> /content/log.txt &


In [29]:
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared
!./cloudflared tunnel --url http://localhost:8501


2025-08-25T03:56:24Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-08-25T03:56:24Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-08-25T03:56:27Z INF +--------------------------------------------------------------------------------------------+
2025-08-25T03:56:27Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-08-25T03:56:27Z INF |  https://replica-acne-doctor-likewise.trycloudflare.co